In [83]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score,classification_report,roc_auc_score, roc_curve
#from sklearn.datasets import load_boston
#from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree

import matplotlib.pyplot as plt


In [84]:
pwd()

'C:\\Users\\kandanulus\\MyTestlab\\BX'

In [85]:
import os

In [86]:
os.chdir('C:\\Users\\kandanulus\\MyTestlab\\BX')

In [87]:
pwd()

'C:\\Users\\kandanulus\\MyTestlab\\BX'

In [88]:
# Read all the privided files with encoding with ISO-8859-1
BXU = pd.read_csv('BXUsers.csv', encoding = "ISO-8859-1", engine='python', na_values={"Age" : [0], "Team" : ['na']})
BXB = pd.read_csv('BXBooks.csv',encoding = "ISO-8859-1", engine='python', nrows =10000)
BXBR = pd.read_csv('BXBookRatings.csv',encoding = "ISO-8859-1", engine='python')
BXR = pd.read_csv('Recommend.csv',encoding = "ISO-8859-1", engine='python')

In [89]:
# Read first five records from BX-Users file contains the information of users.
BXU.head()

,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [90]:
# This file contains the Books are identified by their respective ISBNs
BXB.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [91]:
# BX-Book-Ratings contains ratings given by the user ID and the respective ISBN number.
BXBR.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [92]:
# Books recommentation file contains the recommendations of the book.
BXR.head()

,196,242,3,881250949
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [14]:
# Perform EDA 

# Identify null values in each file.
BXU.isnull().sum()

user_id          0
Location         1
Age         111179
dtype: int64

In [15]:
BXB.isnull().sum()


isbn                   0
book_title             0
book_author            0
year_of_publication    0
publisher              0
dtype: int64

In [16]:
BXBR.isnull().sum()


user_id    0
isbn       0
rating     0
dtype: int64

In [17]:
BXR.isnull().sum()

196          0
242          0
3            0
881250949    0
dtype: int64

In [18]:
# Only the Boosk user file has null values and Clean up NaN values
BXU ['Location'].fillna('Text', inplace=True)

In [20]:
#After replacing number values
BXU.isnull().sum()

user_id          0
Location         0
Age         111179
dtype: int64

In [21]:
BXU ['Age'].fillna(0, inplace=True)

In [22]:
BXU.head()

,user_id,Location,Age
0,1,"nyc, new york, usa",0.0
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",0.0
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",0.0


In [24]:
#Read the data where ratings are given by users
BXBR.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [26]:
# Read the data where ratings are given by users
BXBR['rating'].value_counts()

0     651330
8      95037
10     71814
7      70018
9      62101
5      46953
6      33753
4       8031
3       5445
2       2492
1       1601
Name: rating, dtype: int64

In [52]:
# Joining te data set 
df = pd.merge(BXBR,BXB, on = 'isbn')
df.head()

# BR = (BXBR.set_index("isbn").join(BXB.set_index("isbn"),how="left")).join

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books


In [44]:
df.shape

(245239, 7)

In [45]:
# Number of the unique user.
n_users = df['user_id'].nunique()

In [46]:
print("Number of users: {}".format(n_users))

Number of users: 44608


In [49]:
# Number of the unique books.
n_books= df['isbn'].nunique

In [50]:
print("Number of books: {}".format(n_books))

Number of books: <bound method IndexOpsMixin.nunique of 0         034545104X
1         034545104X
2         034545104X
3         034545104X
4         034545104X
             ...    
245234     312974620
245235     312974620
245236     152024999
245237     440500095
245238     440500095
Name: isbn, Length: 245239, dtype: object>


# Convert the user_id variable to numeric numbers in the correct order.

In [55]:
# convert user_id into the numeric number.
list_userid = df.user_id.unique()
print("length of User list: ", len(list_userid))

length of User list:  44608


In [54]:
def userid_numeric(user_id):
    itemindex = np.where(list_userid==user_id)
    return itemindex[0][0]

In [57]:
list_isbn = df.isbn.unique()
print("length of isbn list:", len(list_isbn))

length of isbn list: 9993


In [58]:
list_isbn

array(['034545104X', '038550120X', '425115801', ..., '312974620',
       '152024999', '440500095'], dtype=object)

In [60]:
def isbn_numeric(isbn):
    itemindex = np.where(list_isbn == isbn)
    return itemindex[0][0]

# Convert both user_id and ISBN to the ordered list, i.e., from 0...n-1

In [61]:
df['user_id_order'] = df['user_id'].apply(userid_numeric)

In [63]:
df['isbn_order'] = df['isbn'].apply(isbn_numeric)

In [64]:
df.head()

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher,user_id_order,isbn_order
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,0,0
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,1,0
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,2,0
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,3,0
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,4,0


# Re-index the columns to build a matrix.

In [66]:
ordered_col = ['user_id_order', 'isbn_order', 'rating', 'book_title', 'book_author', 'year_of_publication','publisher',
               'user_id', 'isbn']
df = df.reindex(columns = ordered_col)

In [67]:
df.head()

,user_id_order,isbn_order,rating,book_title,book_author,year_of_publication,publisher,user_id,isbn
0,0,0,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,276725,034545104X
1,1,0,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,2313,034545104X
2,2,0,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,6543,034545104X
3,3,0,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,8680,034545104X
4,4,0,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,10314,034545104X


# Split your data into two sets (training and testing).

In [68]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.30, random_state = 10)

In [69]:
train.shape

(171667, 9)

In [70]:
test.shape

(73572, 9)

# Make predictions based on user and item variables.

In [82]:
train_matrix = np.zeros((n_users, n_books))
for line in train.itertuples():
    train_matrix [line[1]-1, line[2]-1] = line[3]
    
test_matrix = np.zeros((n_users, n_books))
for line in test.itertuples():
    test_matrix[line[1]-1, line[2]-1] = line[3]

TypeError: 'method' object cannot be interpreted as an integer

In [76]:

from sklearn.metrics.pairwise import pairwise_distances 
user_correlation = pairwise_distances(train_matrix, metric= 'cosine')
item_correlation = pairwise_distances(train_matrix.T, metric= 'cosine')

NameError: name 'train_matrix' is not defined

In [77]:
user_correlation

NameError: name 'user_correlation' is not defined

In [78]:
def predict(ratings,correlation, type= 'user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        rating_diff = (ratings - mean_user_rating[:,np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + correlation.dot(rating_diff) / np.array([np.abs(correlation).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(correlation) / np.array([np.abs(correlation).sum(axis=1)])
        
    return pred 

In [79]:
item_prediction.shape

NameError: name 'item_prediction' is not defined

# Use RMSE to evaluate the predictions.

In [80]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, actual):
    prediction = prediction[actual.nonzero()].flatten() 
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, actual))

In [81]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_matrix)))

NameError: name 'user_prediction' is not defined